#### Definição do agente   

Este agente será um assistente para estudos, focado em leitura de papers.   
O propósito será explicar em linguagem simples o que está descrito no paper.



#### palavras chaves:  
Aula 1: "LANGCHAIN"

In [0]:
!pip install -q --upgrade -r requirements.txt

In [0]:
import json
import prompts
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
from langchain_core.messages import SystemMessage, HumanMessage

GEMINI_API_KEY = json.load(open("config.json"))["GEMINI_API_KEY"]

In [0]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature = 0, api_key = GEMINI_API_KEY)

In [0]:
class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
    campos_faltantes: List[str] = Field(default_factory = list)

In [0]:
llm_triagem = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature = 0, api_key = GEMINI_API_KEY)

In [0]:
triagem_prompt = """Você é o recepcionista virtual de uma sistema de estudo online chamado ReadMyPaper. Você deve fazer o primeiro atendimento dos usuários identificando se os requisitos para iniciar a leitura do paper foram cumpridos.
Dada a mensagem do usuário, retorne SOMENTE um JSON com:
'''{
"decisao": "TUDO_CERTO" | "PEDIR_INFO" | "ABRIR_CHAMADO",
"urgencia": "BAIXA" | "MEDIA" | "ALTA",
"campos_faltantes": ["..."]
}'''
Regras:
- **TUDO_CERTO**: Perguntas claras sobre o conteúdo do paper, ou pedido de ajuda para enviar o paper para análise (Ex: "Qual o principal assunto do paper?", "Qual a metodologia utilizada?", "Como faço para enviar o paper?").
- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o paper (Ex: "Preciso de ajuda", "Tenho uma dúvida geral", "Qualquer pergunta fora do contexto do ReadMyPaper").
- **ABRIR_CHAMADO**: Situações em que não é possível continuar com a atendimento com o usuário. (Ex: "O usuáiro afirma que já fez o envio do paper mas ainda não há conteúdo para ser avaliado.", "O usuário está com problemas ao tentar enviar o paper", "O usuário pede abrir chamado").
Analise a mensagem e decida a ação mais apropriada.
"""

In [0]:
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def get_papers_list():
    """
    
    """
    pass
          

def check_paper_exist(paper_name) -> list:
    """
    
    """
    if os.path.exists(f"papers_to_read"):
        return True
    return os.path.exists(f"papers/{paper_name}.pdf")
    


def triagem(message: str) -> dict:
    """
    Retorna um dicionário com as seguintes chaves:
    - decisao: "TUDO_CERTO", "PEDIR_INFO", "ABRIR_CHAMADO"
    - urgencia: "BAIXA", "MEDIA", "ALTA"
    - campos_faltantes: lista de campos faltantes
    """
       
    triagem_out: TriagemOut = triagem_chain.invoke([SystemMessage(content = triagem_prompt), HumanMessage(content = message)])

    return triagem_out.model_dump()  

In [0]:
triagem("Quero iniciar a leitura do paper.")